### RAG Pipelines - Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

c:\Users\chanc\anaconda3\envs\ragproj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 2 PDF files to process

Processing: Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf
  ✓ Loaded 8 pages

Processing: Team 28-VLM Hallucination.pdf
  ✓ Loaded 8 pages

Total documents loaded: 16


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'Adobe PDF Library 9.0', 'creator': 'Acrobat PDFMaker 9.0 for Word', 'creationdate': '2024-02-06T15:08:55+05:30', 'author': 'Ahilya Bandgar', 'moddate': '2024-02-06T15:08:58+05:30', 'sourcemodified': 'D:20240206093818', 'rgid': 'PB:379933077_AS:11431281256064873@1719493068149', 'source': '..\\data\\pdf\\Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': 'Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf', 'file_type': 'pdf'}, page_content="See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/379933077\nImproving Earth Observations by correlating Multiple Satellite Data: A\nComparative Analysis of Landsat, MODIS and Sentinel

In [4]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [5]:
chunks=split_documents(all_pdf_documents)
chunks

Split 16 documents into 85 chunks

Example chunk:
Content: See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/379933077
Improving Earth Observations by correlating Multiple Satellite Data: A
Comparati...
Metadata: {'producer': 'Adobe PDF Library 9.0', 'creator': 'Acrobat PDFMaker 9.0 for Word', 'creationdate': '2024-02-06T15:08:55+05:30', 'author': 'Ahilya Bandgar', 'moddate': '2024-02-06T15:08:58+05:30', 'sourcemodified': 'D:20240206093818', 'rgid': 'PB:379933077_AS:11431281256064873@1719493068149', 'source': '..\\data\\pdf\\Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': 'Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Adobe PDF Library 9.0', 'creator': 'Acrobat PDFMaker 9.0 for Word', 'creationdate': '2024-02-06T15:08:55+05:30', 'author': 'Ahilya Bandgar', 'moddate': '2024-02-06T15:08:58+05:30', 'sourcemodified': 'D:20240206093818', 'rgid': 'PB:379933077_AS:11431281256064873@1719493068149', 'source': '..\\data\\pdf\\Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': 'Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf', 'file_type': 'pdf'}, page_content="See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/379933077\nImproving Earth Observations by correlating Multiple Satellite Data: A\nComparative Analysis of Landsat, MODIS and Sentinel

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 598.85it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully. Embedding dimension: 384


### VectorStore

In [8]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [9]:
chunks

[Document(metadata={'producer': 'Adobe PDF Library 9.0', 'creator': 'Acrobat PDFMaker 9.0 for Word', 'creationdate': '2024-02-06T15:08:55+05:30', 'author': 'Ahilya Bandgar', 'moddate': '2024-02-06T15:08:58+05:30', 'sourcemodified': 'D:20240206093818', 'rgid': 'PB:379933077_AS:11431281256064873@1719493068149', 'source': '..\\data\\pdf\\Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': 'Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf', 'file_type': 'pdf'}, page_content="See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/379933077\nImproving Earth Observations by correlating Multiple Satellite Data: A\nComparative Analysis of Landsat, MODIS and Sentinel

In [10]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 85 texts...


Batches: 100%|██████████| 3/3 [00:01<00:00,  1.51it/s]

Generated embeddings with shape: (85, 384)
Adding 85 documents to vector store...
Successfully added 85 documents to vector store
Total documents in collection: 85


### Retriever Pipeline From VectorStore

In [11]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



    

In [12]:
rag_retriever

In [14]:
rag_retriever.retrieve("What is the image captioning research paper explaining?")

Retrieving documents for query: 'What is the image captioning research paper explaining?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 65.53it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_254c49a3_84',
  'content': '[13] Image Captioning Evaluation in the Age of Multimodal LLMs , arXiv 2025.\n[14] Pseudo Content Hallucination for Unpaired Image Captioning , ACM MM 2024.\n[15] Hyperbolic Learning with Synthetic Captions , CVPR 2024.\nProject Repository\nThe complete code and resources for this project can be found on\nGitHub:\nhttps://github.com/Avipsa-Bhujabal/Image𝐶𝑎𝑝𝑡𝑖𝑜𝑛𝑖𝑛𝑔 𝐺𝑒𝑛𝑒𝑟𝑎𝑡𝑖𝑜𝑛',
  'metadata': {'content_length': 371,
   'subject': '',
   'producer': 'pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0',
   'creator': 'LaTeX with acmart 2024/12/28 v2.12 Typesetting articles for the Association for Computing Machinery and hyperref 2024-01-20 v7.01h Hypertext links for LaTeX',
   'creationdate': '2025-05-10T14:29:11+00:00',
   'trapped': '/False',
   'total_pages': 8,
   'page_label': '8',
   'doc_index': 84,
   'source_file': 'Team 28-VLM Hallucination.pdf',
   'moddate': '2025-05-10T14:29:11+00:00',
   'file_type': 'pdf',


In [15]:
rag_retriever.retrieve("What is the conclusion on flood mapping paper?")


Retrieving documents for query: 'What is the conclusion on flood mapping paper?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 66.69it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_fc120cf0_37',
  'content': 'monitoring of changes in the landscape related to flooding. \nThese maps provide valuable information for emergency \nresponse, risk assessment, and long-term flood management. \nThe objective of this research is to get the difference map \nbetween multiple satellite data in combination, so that an \naccurate flood extent area would help to get the users in the \nfields of early warning systems, flood management, \nresearchers, etc. But, certain limitations such as limited \nspectral information, land cover changes or lack of data \naccessibility may lead to the discrepancies in the results.  \nR\nEFERENCES \n[1] Nsangou D, Kpoumié A, Mfonka Z, Bateni SM, Ngouh AN, Ndam          \nNgoupayou JR, “The Mfoundi Watershed at Yaoundé in the \nHumid Tropical Zone of Camer oon: a case study of urban food \nsusceptibility mapping,” Earth Systems and Environment 2021. \n[2] Asinya EA, Alam MJB, “Flood risk in rivers: climate driven or \nmorphological adju

### RAG Pipeline- VectorDB To LLM Output Generation

In [17]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

In [19]:
class GroqLLM:
    def __init__(self, model_name: str = "llama-3.1-8b-instant", api_key: str =None):
        """
        Initialize Groq LLM
        
        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        
        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )
        
        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)
        
        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
    


In [20]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None

Initialized Groq LLM with model: llama-3.1-8b-instant
Groq LLM initialized successfully!


In [22]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("What is the image captioning research paper explaining?")

Retrieving documents for query: 'What is the image captioning research paper explaining?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 77.00it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_254c49a3_84',
  'content': '[13] Image Captioning Evaluation in the Age of Multimodal LLMs , arXiv 2025.\n[14] Pseudo Content Hallucination for Unpaired Image Captioning , ACM MM 2024.\n[15] Hyperbolic Learning with Synthetic Captions , CVPR 2024.\nProject Repository\nThe complete code and resources for this project can be found on\nGitHub:\nhttps://github.com/Avipsa-Bhujabal/Image𝐶𝑎𝑝𝑡𝑖𝑜𝑛𝑖𝑛𝑔 𝐺𝑒𝑛𝑒𝑟𝑎𝑡𝑖𝑜𝑛',
  'metadata': {'moddate': '2025-05-10T14:29:11+00:00',
   'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0',
   'subject': '',
   'page_label': '8',
   'creationdate': '2025-05-10T14:29:11+00:00',
   'source_file': 'Team 28-VLM Hallucination.pdf',
   'producer': 'pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0',
   'doc_index': 84,
   'trapped': '/False',
   'source': '..\\data\\pdf\\Team 28-VLM Hallucination.pdf',
   'page': 7,
   'content_length': 371,
   'file_type': 'pdf',
   'titl

### Integration Vectordb Context pipeline With LLM output

In [23]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [24]:
answer=rag_simple("What is the image captioning research paper explaining?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is the image captioning research paper explaining?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 53.03it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


The research papers explain various aspects of image captioning, including:

- Evaluating image captioning in the age of multimodal LLMs (Large Language Models) ([13] Image Captioning Evaluation in the Age of Multimodal LLMs, arXiv 2025).
- Generating improved image captions using pseudo content hallucination for unpaired image captioning ([14] Pseudo Content Hallucination for Unpaired Image Captioning, ACM MM 2024).
- Using hyperbolic learning with synthetic captions for image captioning ([15] Hyperbolic Learning with Synthetic Captions, CVPR 2024).

These papers aim to improve the quality and accuracy of image captioning models.


### Testing

In [35]:
# Check if PDFs are actually being found
from pathlib import Path

pdf_dir = Path("../data")
pdf_files = list(pdf_dir.glob("**/*.pdf"))
print(f"Found {len(pdf_files)} PDF files:")
for f in pdf_files:
    print(f"  - {f}")

Found 2 PDF files:
  - ..\data\pdf\Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf
  - ..\data\pdf\Team 28-VLM Hallucination.pdf


In [36]:
# Check if documents are in ChromaDB
print(f"Documents in vector store: {vectorstore.collection.count()}")
if vectorstore.collection.count() == 0:
    print("❌ Vector store is EMPTY! Documents not being added.")
else:
    print(f"✓ Vector store has {vectorstore.collection.count()} documents")

Documents in vector store: 85
✓ Vector store has 85 documents


In [37]:
# Test retrieval with lower threshold to see what's being found
query = "What was the objective of image captioning research paper?"
results = rag_retriever.retrieve(query, top_k=5, score_threshold=0.0)  # Set to 0 to see all results

print(f"Retrieved {len(results)} documents:")
for i, doc in enumerate(results):
    print(f"\n[{i+1}] Similarity Score: {doc['similarity_score']:.4f}")
    print(f"Content: {doc['content'][:200]}...")
    print(f"Metadata: {doc['metadata']}")

Retrieving documents for query: 'What was the objective of image captioning research paper?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 30.86it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)
Retrieved 1 documents:

[1] Similarity Score: 0.0499
Content: [13] Image Captioning Evaluation in the Age of Multimodal LLMs , arXiv 2025.
[14] Pseudo Content Hallucination for Unpaired Image Captioning , ACM MM 2024.
[15] Hyperbolic Learning with Synthetic Capt...
Metadata: {'page_label': '8', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'doc_index': 84, 'moddate': '2025-05-10T14:29:11+00:00', 'content_length': 371, 'file_type': 'pdf', 'title': 'Comparative Analysis of ML Models for Automatic Image Captioning', 'subject': '', 'total_pages': 8, 'producer': 'pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'creator': 'LaTeX with acmart 2024/12/28 v2.12 Typesetting articles for the Association for Computing Machinery and hyperref 2024-01-20 v7.01h Hypertext links for LaTeX', 'source_file': 'Team 28-VLM Hallucinat

In [38]:
# Check first few chunks that were created
print(f"Total chunks created: {len(chunks)}")
if len(chunks) > 0:
    print(f"\nFirst chunk content preview:")
    print(chunks[0].page_content[:300])
    print(f"\nFirst chunk metadata:")
    print(chunks[0].metadata)
else:
    print("❌ No chunks created! Check if PDFs are loaded.")

Total chunks created: 85

First chunk content preview:
See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/379933077
Improving Earth Observations by correlating Multiple Satellite Data: A
Comparative Analysis of Landsat, MODIS and Sentinel Satellite Data for Flood
Mapping
Conference Paper · Febru

First chunk metadata:
{'producer': 'Adobe PDF Library 9.0', 'creator': 'Acrobat PDFMaker 9.0 for Word', 'creationdate': '2024-02-06T15:08:55+05:30', 'author': 'Ahilya Bandgar', 'moddate': '2024-02-06T15:08:58+05:30', 'sourcemodified': 'D:20240206093818', 'rgid': 'PB:379933077_AS:11431281256064873@1719493068149', 'source': '..\\data\\pdf\\Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparative-Analysis-of-Landsat-MODIS-and-Sentinel-Satellite-Data-for-Flood-Mapping.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': 'Improving-Earth-Observations-by-correlating-Multiple-Satellite-Data-A-Comparati

### Enhanced RAG Pipeline Features

In [44]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("Summary of VLM Hallucination paper", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'Summary of VLM Hallucination paper'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 90.84it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
Answer: No relevant context found.
Sources: []
Confidence: 0.0
Context Preview: 


In [34]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("What was the objective of image captioning research paper?", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'What was the objective of image captioning research paper?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 62.19it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)



Final Answer: No relevant context found.
Summary: There is no information to summarize as the provided answer is a statement indicating that no relevant context was found.
History: {'question': 'What was the objective of image captioning research paper?', 'answer': 'No relevant context found.', 'sources': [], 'summary': 'There is no information to summarize as the provided answer is a statement indicating that no relevant context was found.'}
